# Deep dive — groupby() + aggregation (Pandas)
## Big idea — Split → Apply → Combine

#### groupby() does exactly three things:

Split the DataFrame into groups based on key(s) (column values, functions, or index).

Apply a function to each group (aggregate, transform, custom).

Combine the results back into a DataFrame/Series.

This pattern powers most summary/summary-by operations: totals, averages, counts, top-N per group, rolling stats per group, etc.

In [2]:
import pandas as pd

data = {
    'Department': ['IT','HR','IT','HR','Finance','IT','Finance'],
    'Employee':  ['Pavan','Ravi','Anu','Sneha','Arjun','Riya','Divya'],
    'Salary':    [50000,45000,52000,48000,55000,53000,60000],
    'Experience':[2,3,4,5,3,2,6]
}
df = pd.DataFrame(data)


In [3]:
df

,Department,Employee,Salary,Experience
0,IT,Pavan,50000,2
1,HR,Ravi,45000,3
2,IT,Anu,52000,4
3,HR,Sneha,48000,5
4,Finance,Arjun,55000,3
5,IT,Riya,53000,2
6,Finance,Divya,60000,6


### Mean salary by department:

What happens:

Groups created for 'Finance', 'HR', 'IT'.

For each group Pandas computes mean of Salary.

Result is a Series with group labels as index.

In [4]:
df.groupby('Department')['Salary'].mean()
### Mean salary by department:


Department
Finance    57500.000000
HR         46500.000000
IT         51666.666667
Name: Salary, dtype: float64

###  Aggregating multiple columns

In [5]:
df.groupby('Department')[['Salary','Experience']].mean().max()


Salary        57500.0
Experience        4.5
dtype: float64

4) Multiple aggregations: .agg() and .aggregate()

.agg() accepts:

a single function ('mean')

a list of functions (['mean','sum'])

a dict mapping column → function(s)

named aggregation (since pandas 0.25+), which produces clean column names

In [6]:
# same agg for multiple numeric columns (exclude non-numeric 'Employee' column)
df.groupby('Department')[['Salary','Experience']].agg(['mean','max'])

# different aggs per column (apply numeric aggs only to numeric columns)
# df.groupby('Department').agg({'Salary':['mean','max'], 'Experience':'sum'})


Salary        Experience    
                    mean    max       mean max
Department                                    
Finance     57500.000000  60000   4.500000   6
HR          46500.000000  48000   4.000000   5
IT          51666.666667  53000   2.666667   4

### Named aggregation (cleaner column names):

In [7]:
df.groupby('Department').agg(
    avg_salary = ('Salary','mean'),
    max_salary = ('Salary','max'),
    total_experience = ('Experience','sum')
).reset_index()


,Department,avg_salary,max_salary,total_experience
0,Finance,57500.000000,60000,9
1,HR,46500.000000,48000,8
2,IT,51666.666667,53000,8


 ### 5) .size() vs .count()

groupby.size() → counts rows per group (including NaNs in other columns). Returns Series.

groupby['col'].count() → counts non-null values of col per group.

In [8]:
df.groupby('Department').size()




Department
Finance    2
HR         2
IT         3
dtype: int64

In [9]:
df.groupby('Department')['Salary'].count()

Department
Finance    2
HR         2
IT         3
Name: Salary, dtype: int64

###   6 as_index and reset_index

By default groupby() returns group labels as the index. Use reset_index() or as_index=False to keep them as columns.

In [10]:
# df.groupby('Department', as_index=False)['Salary'].mean()
# OR
df.groupby('Department')['Salary'].mean().reset_index()


,Department,Salary
0,Finance,57500.000000
1,HR,46500.000000
2,IT,51666.666667


### Group by multiple keys

In [11]:
df.groupby(['Department','Experience'])['Salary'].mean()


Department  Experience
Finance     3             55000.0
            6             60000.0
HR          3             45000.0
            5             48000.0
IT          2             51500.0
            4             52000.0
Name: Salary, dtype: float64

###  Iterating groups
 
name is group label; group is a DataFrame with rows in that group.

In [12]:
g = df.groupby('Department')
for name, group in g:
    print(name)
    print(group)


Finance
  Department Employee  Salary  Experience
4    Finance    Arjun   55000           3
6    Finance    Divya   60000           6
HR
  Department Employee  Salary  Experience
1         HR     Ravi   45000           3
3         HR    Sneha   48000           5
IT
  Department Employee  Salary  Experience
0         IT    Pavan   50000           2
2         IT      Anu   52000           4
5         IT     Riya   53000           2


21) Short summary

groupby() is the core tool for summarizing by groups.

Use .agg() / named aggregation for clean multi-column summaries.

Use .transform() to broadcast group-level results back to rows.

Use .apply() for group-wise custom ops when necessary.

Grouper is your friend for time-series grouping.

pivot_table offers alternative with easy reshape.

In [13]:
df.groupby('Department').apply(lambda x: x['Salary'].mean() + x['Experience'].sum())

C:\Users\pavan\AppData\Local\Temp\ipykernel_15180\678865482.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Department').apply(lambda x: x['Salary'].mean() + x['Experience'].sum())


Department
Finance    57509.000000
HR         46508.000000
IT         51674.666667
dtype: float64

In [14]:
df

,Department,Employee,Salary,Experience
0,IT,Pavan,50000,2
1,HR,Ravi,45000,3
2,IT,Anu,52000,4
3,HR,Sneha,48000,5
4,Finance,Arjun,55000,3
5,IT,Riya,53000,2
6,Finance,Divya,60000,6


In [15]:
df["avg_salary"]=df.groupby("Department")["Salary"].mean()

In [16]:
df

,Department,Employee,Salary,Experience,avg_salary
0,IT,Pavan,50000,2,NaN
1,HR,Ravi,45000,3,NaN
2,IT,Anu,52000,4,NaN
3,HR,Sneha,48000,5,NaN
4,Finance,Arjun,55000,3,NaN
5,IT,Riya,53000,2,NaN
6,Finance,Divya,60000,6,NaN


In [17]:
df.groupby(["Department" ,"Employee"])[["Salary","Experience"]].sum()

Salary  Experience
Department Employee                    
Finance    Arjun      55000           3
           Divya      60000           6
HR         Ravi       45000           3
           Sneha      48000           5
IT         Anu        52000           4
           Pavan      50000           2
           Riya       53000           2

In [18]:
df

,Department,Employee,Salary,Experience,avg_salary
0,IT,Pavan,50000,2,NaN
1,HR,Ravi,45000,3,NaN
2,IT,Anu,52000,4,NaN
3,HR,Sneha,48000,5,NaN
4,Finance,Arjun,55000,3,NaN
5,IT,Riya,53000,2,NaN
6,Finance,Divya,60000,6,NaN


In [19]:
df["Salary"].var()

np.float64(23809523.80952381)

In [20]:
df.groupby("Salary").last()

,Department,Employee,Experience,avg_salary
Salary,,,,
45000,HR,Ravi,3,NaN
48000,HR,Sneha,5,NaN
50000,IT,Pavan,2,NaN
52000,IT,Anu,4,NaN
53000,IT,Riya,2,NaN
55000,Finance,Arjun,3,NaN
60000,Finance,Divya,6,NaN
